In [72]:
#Importing the Libraries
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import io

In [38]:
# data_csv = pd.read_csv('./NSL-KDD/KDDTrain+_20Percent.csv')
from google.colab import files
uploaded = files.upload()

Saving KDDTrain+.csv to KDDTrain+ (1).csv


In [39]:
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
# df = pd.read_csv(io.StringIO(uploaded['KDDTrain+_20Percent.csv'].decode('utf-8')), sep=",", names=feature)
df = pd.read_csv(io.StringIO(uploaded['KDDTrain+.csv'].decode('utf-8')), sep=",", names=feature)
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,id,'duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',...,'dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','class'
1,1,0,tcp,ftp_data,SF,491,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
2,2,0,udp,other,SF,146,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
3,3,0,tcp,private,S0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
4,4,0,tcp,http,SF,232,8153,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125969,125969,0,tcp,private,S0,0,0,0,0,0,...,25,0.1,0.06,0.0,0.0,1.0,1.0,0.0,0.0,anomaly
125970,125970,8,udp,private,SF,105,145,0,0,0,...,244,0.96,0.01,0.01,0.0,0.0,0.0,0.0,0.0,normal
125971,125971,0,tcp,smtp,SF,2231,384,0,0,0,...,30,0.12,0.06,0.0,0.0,0.72,0.0,0.01,0.0,normal
125972,125972,0,tcp,klogin,S0,0,0,0,0,0,...,8,0.03,0.05,0.0,0.0,1.0,1.0,0.0,0.0,anomaly


In [40]:
df = df.drop(0)
df = df.reset_index(drop=True)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   duration                     125973 non-null  object
 1   protocol_type                125973 non-null  object
 2   service                      125973 non-null  object
 3   flag                         125973 non-null  object
 4   src_bytes                    125973 non-null  object
 5   dst_bytes                    125973 non-null  object
 6   land                         125973 non-null  object
 7   wrong_fragment               125973 non-null  object
 8   urgent                       125973 non-null  object
 9   hot                          125973 non-null  object
 10  num_failed_logins            125973 non-null  object
 11  logged_in                    125973 non-null  object
 12  num_compromised              125973 non-null  object
 13  root_shell    

In [42]:
df.describe().T

,count,unique,top,freq
duration,125973,125973,1,1
protocol_type,125973,3137,0,100888
service,125973,3,tcp,102689
flag,125973,70,http,40338
src_bytes,125973,11,SF,74945
dst_bytes,125973,4496,0,42912
land,125973,11716,0,59095
wrong_fragment,125973,4,0,109567
urgent,125973,6,0,108641
hot,125973,6,0,109582


In [43]:
df.shape

(125973, 43)

In [44]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,1,0,tcp,ftp_data,SF,491,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,2,0,udp,other,SF,146,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,3,0,tcp,private,S0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,4,0,tcp,http,SF,232,8153,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,5,0,tcp,http,SF,199,420,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [45]:
X_train = df.iloc[:, 1:41]
X_train

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00
3,0,tcp,http,SF,232,8153,0,0,0,0,...,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00
4,0,tcp,http,SF,199,420,0,0,0,0,...,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.0,255,25,0.1,0.06,0.0,0.0,1.0,1.0,0.0
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.0,255,244,0.96,0.01,0.01,0.0,0.0,0.0,0.0
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.0,255,30,0.12,0.06,0.0,0.0,0.72,0.0,0.01
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.0,255,8,0.03,0.05,0.0,0.0,1.0,1.0,0.0


In [46]:
X_train.columns

Index(['protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
       'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
       'num_compromised', 'root_shell', 'su_attempted', 'num_root',
       'num_file_creations', 'num_shells', 'num_access_files',
       'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
       'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
       'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
       'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
       'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate',
       'dst_host_rerror_rate', 'dst_host_srv_rerror_rate'],
      dtype='object')

In [47]:
# Label Encoding the data in all the features
print(X_train['service'].unique())
print(X_train['flag'].unique())
print(X_train['src_bytes'].unique())

le = LabelEncoder()
le.fit(X_train['service'])
X_train['service'] = le.transform(X_train['service'])

le = LabelEncoder()
le.fit(X_train['flag'])
X_train['flag'] = le.transform(X_train['flag'])

le = LabelEncoder()
le.fit(X_train['src_bytes'])
X_train['src_bytes'] = le.transform(X_train['src_bytes'])

['tcp' 'udp' 'icmp']
['ftp_data' 'other' 'private' 'http' 'remote_job' 'name' 'netbios_ns'
 'eco_i' 'mtp' 'telnet' 'finger' 'domain_u' 'supdup' 'uucp_path' 'Z39_50'
 'smtp' 'csnet_ns' 'uucp' 'netbios_dgm' 'urp_i' 'auth' 'domain' 'ftp'
 'bgp' 'ldap' 'ecr_i' 'gopher' 'vmnet' 'systat' 'http_443' 'efs' 'whois'
 'imap4' 'iso_tsap' 'echo' 'klogin' 'link' 'sunrpc' 'login' 'kshell'
 'sql_net' 'time' 'hostnames' 'exec' 'ntp_u' 'discard' 'nntp' 'courier'
 'ctf' 'ssh' 'daytime' 'shell' 'netstat' 'pop_3' 'nnsp' 'IRC' 'pop_2'
 'printer' 'tim_i' 'pm_dump' 'red_i' 'netbios_ssn' 'rje' 'X11' 'urh_i'
 'http_8001' 'aol' 'http_2784' 'tftp_u' 'harvest']
['SF' 'S0' 'REJ' 'RSTR' 'SH' 'RSTO' 'S1' 'RSTOS0' 'S3' 'S2' 'OTH']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [48]:
# # Label Encoding the data in all the features
# from sklearn.compose import make_column_transformer
# from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import OneHotEncoder

# categorical_columns = ['service', 'flag', 'src_bytes']
# numerical_columns = ['protocol_type', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted'
# , 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate','srv_serror_rate', 'same_srv_rate',
# 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
# 'dst_host_serror_rate','dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

# column_trans = make_column_transformer(
#     [
#      (categorical_columns, OneHotEncoder(dtype='int')),
#       (numerical_columns, RobustScaler())
#     ], remainder='drop')

# column_trans.fit_transform(X_train)
X_train.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,20,9,491,0,0,0,0,0,...,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05
1,0,2,44,9,146,0,0,0,0,0,...,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00
2,0,1,49,5,0,0,0,0,0,0,...,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00
3,0,1,24,9,232,8153,0,0,0,0,...,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00
4,0,1,24,9,199,420,0,0,0,0,...,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00


In [49]:
Y_train = df.iloc[:,42:]
Y_train

,difficulty
0,normal
1,normal
2,anomaly
3,normal
4,normal
...,...
125968,anomaly
125969,normal
125970,normal
125971,anomaly


In [50]:
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_train

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 0, ..., 1, 0, 1])

In [51]:
X_train.shape, Y_train.shape

((125973, 40), (125973,))

In [52]:
#Loading Testing data
from google.colab import files
uploaded = files.upload()

Saving KDDTest+.csv to KDDTest+ (1).csv


In [53]:
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
# df = pd.read_csv(io.StringIO(uploaded['KDDTrain+_20Percent.csv'].decode('utf-8')), sep=",", names=feature)
df = pd.read_csv(io.StringIO(uploaded['KDDTest+.csv'].decode('utf-8')), sep=",", names=feature)
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty
0,id,'duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',...,'dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate','class'
1,1,0,tcp,private,REJ,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
2,2,0,tcp,private,REJ,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.00,1.00,1.00,anomaly
3,3,2,tcp,ftp_data,SF,12983,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.00,0.00,0.00,normal
4,4,0,icmp,eco_i,SF,20,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.00,0.00,0.00,anomaly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22540,22540,0,tcp,smtp,SF,794,333,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.0,0.0,0.0,normal
22541,22541,0,tcp,http,SF,317,938,0,0,0,...,255,1.0,0.0,0.01,0.01,0.01,0.0,0.0,0.0,normal
22542,22542,0,tcp,http,SF,54540,8314,0,0,0,...,255,1.0,0.0,0.0,0.0,0.0,0.0,0.07,0.07,anomaly
22543,22543,0,udp,domain_u,SF,42,42,0,0,0,...,252,0.99,0.01,0.0,0.0,0.0,0.0,0.0,0.0,normal


In [54]:
df = df.drop(0)
df = df.reset_index(drop=True)

In [55]:
X_test = df.iloc[:, 1:41]
X_test

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,255,10,0.04,0.06,0.00,0.00,0.00,0.00,1.00
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,255,1,0.00,0.06,0.00,0.00,0.00,0.00,1.00
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.00,134,86,0.61,0.04,0.61,0.02,0.00,0.00,0.00
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,3,57,1.00,0.00,1.00,0.28,0.00,0.00,0.00
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.75,29,86,0.31,0.17,0.03,0.02,0.00,0.00,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.0,100,141,0.72,0.06,0.01,0.01,0.01,0.0,0.0
22540,0,tcp,http,SF,317,938,0,0,0,0,...,0.18,197,255,1.0,0.0,0.01,0.01,0.01,0.0,0.0
22541,0,tcp,http,SF,54540,8314,0,0,0,2,...,0.2,255,255,1.0,0.0,0.0,0.0,0.0,0.0,0.07
22542,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.33,255,252,0.99,0.01,0.0,0.0,0.0,0.0,0.0


In [56]:
# Label Encoding the data in all the features
print(X_test['service'].unique())
print(X_test['flag'].unique())
print(X_test['src_bytes'].unique())

le = LabelEncoder()
le.fit(X_test['service'])
X_test['service'] = le.transform(X_test['service'])

le = LabelEncoder()
le.fit(X_test['flag'])
X_test['flag'] = le.transform(X_test['flag'])

le = LabelEncoder()
le.fit(X_test['src_bytes'])
X_test['src_bytes'] = le.transform(X_test['src_bytes'])

['tcp' 'icmp' 'udp']
['private' 'ftp_data' 'eco_i' 'telnet' 'http' 'smtp' 'ftp' 'ldap' 'pop_3'
 'courier' 'discard' 'ecr_i' 'imap4' 'domain_u' 'mtp' 'systat' 'iso_tsap'
 'other' 'csnet_ns' 'finger' 'uucp' 'whois' 'netbios_ns' 'link' 'Z39_50'
 'sunrpc' 'auth' 'netbios_dgm' 'uucp_path' 'vmnet' 'domain' 'name' 'pop_2'
 'http_443' 'urp_i' 'login' 'gopher' 'exec' 'time' 'remote_job' 'ssh'
 'kshell' 'sql_net' 'shell' 'hostnames' 'echo' 'daytime' 'pm_dump' 'IRC'
 'netstat' 'ctf' 'nntp' 'netbios_ssn' 'tim_i' 'supdup' 'bgp' 'nnsp' 'rje'
 'printer' 'efs' 'X11' 'ntp_u' 'klogin' 'tftp_u']
['REJ' 'SF' 'RSTO' 'S0' 'RSTR' 'SH' 'S3' 'S2' 'S1' 'RSTOS0' 'OTH']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [57]:
X_test.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,1,45,1,0,0,0,0,0,0,...,0.00,255,10,0.04,0.06,0.00,0.00,0.00,0.00,1.00
1,0,1,45,1,0,0,0,0,0,0,...,0.00,255,1,0.00,0.06,0.00,0.00,0.00,0.00,1.00
2,2,1,19,9,12983,0,0,0,0,0,...,0.00,134,86,0.61,0.04,0.61,0.02,0.00,0.00,0.00
3,0,0,13,9,20,0,0,0,0,0,...,1.00,3,57,1.00,0.00,1.00,0.28,0.00,0.00,0.00
4,1,1,55,2,0,15,0,0,0,0,...,0.75,29,86,0.31,0.17,0.03,0.02,0.00,0.00,0.83


In [58]:
Y_test = df.iloc[:,42:]
le = LabelEncoder()
Y_test = le.fit_transform(Y_test)
Y_test

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 1, ..., 0, 1, 0])

In [59]:
X_test.shape, Y_test.shape

((22544, 40), (22544,))

In [60]:
# AutoEncoder Model Preparation
n_inputs = X_train.shape[1]
# define encoder
input_data_shape= Input(shape=(n_inputs,))
# encoder 1 level 1
encoder= Dense(14)(input_data_shape)
encoder = BatchNormalization()(encoder)
# encoder= LeakyReLU()(encoder)
encoder= sigmoid(encoder)

# encoder 1 level 2
encoder= Dense(28)(encoder)
encoder= BatchNormalization()(encoder)
# encoder= LeakyReLU()(encoder)
encoder= sigmoid(encoder)

# encoder 1 level 3
encoder= Dense(28)(encoder)
encoder= BatchNormalization()(encoder)
# encoder= LeakyReLU()(encoder)
encoder= sigmoid(encoder)

# encoder 2 level 1
encoder= Dense(14)(encoder)
encoder = BatchNormalization()(encoder)
# encoder= LeakyReLU()(encoder)
encoder= sigmoid(encoder)

# encoder 2 level 2
encoder= Dense(28)(encoder)
encoder= BatchNormalization()(encoder)
# encoder= LeakyReLU()(encoder)
encoder= sigmoid(encoder)

# encoder 2 level 3
encoder= Dense(28)(encoder)
encoder= BatchNormalization()(encoder)
# encoder= LeakyReLU()(encoder)
encoder= sigmoid(encoder)


# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(encoder)

# define decoder 1 level 1
decoder = Dense(28)(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = sigmoid(decoder)
# define decoder 1 level 2
decoder = Dense(28)(decoder)
decoder = BatchNormalization()(decoder)
decoder = sigmoid(decoder)
# decoder 1 level 3
decoder = Dense(14)(decoder)
decoder = BatchNormalization()(decoder)
decoder = sigmoid(decoder)

# define decoder 2 level 1
decoder = Dense(28)(decoder)
decoder = BatchNormalization()(decoder)
decoder = sigmoid(decoder)
# define decoder 2 level 2
decoder = Dense(28)(decoder)
decoder = BatchNormalization()(decoder)
decoder = sigmoid(decoder)
# decoder 2 level 3
decoder = Dense(14)(decoder)
decoder = BatchNormalization()(decoder)
decoder = sigmoid(decoder)


In [61]:
# output layer
output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
model = Model(inputs=input_data_shape, outputs=output)
# compile autoencoder model
model.compile(optimizer='adam', loss='mse')

In [62]:
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 40)]              0         
                                                                 
 dense_14 (Dense)            (None, 14)                574       
                                                                 
 batch_normalization_12 (Bat  (None, 14)               56        
 chNormalization)                                                
                                                                 
 tf.math.sigmoid_12 (TFOpLam  (None, 14)               0         
 bda)                                                            
                                                                 
 dense_15 (Dense)            (None, 28)                420       
                                                                 
 batch_normalization_13 (Bat  (None, 28)               112 

In [63]:
Y_train = np.asarray(Y_train).astype(np.float32)
Y_test = np.asarray(Y_test).astype(np.float32)
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

In [64]:
Y_train, Y_test, X_train, X_test

(array([1., 1., 0., ..., 1., 0., 1.], dtype=float32),
 array([0., 0., 1., ..., 0., 1., 0.], dtype=float32),
 array([[0.0e+00, 1.0e+00, 2.0e+01, ..., 0.0e+00, 0.0e+00, 5.0e-02],
        [0.0e+00, 2.0e+00, 4.4e+01, ..., 0.0e+00, 0.0e+00, 0.0e+00],
        [0.0e+00, 1.0e+00, 4.9e+01, ..., 1.0e+00, 1.0e+00, 0.0e+00],
        ...,
        [0.0e+00, 1.0e+00, 5.4e+01, ..., 7.2e-01, 0.0e+00, 1.0e-02],
        [0.0e+00, 1.0e+00, 3.0e+01, ..., 1.0e+00, 1.0e+00, 0.0e+00],
        [0.0e+00, 1.0e+00, 2.0e+01, ..., 0.0e+00, 0.0e+00, 0.0e+00]],
       dtype=float32),
 array([[ 0.  ,  1.  , 45.  , ...,  0.  ,  0.  ,  1.  ],
        [ 0.  ,  1.  , 45.  , ...,  0.  ,  0.  ,  1.  ],
        [ 2.  ,  1.  , 19.  , ...,  0.  ,  0.  ,  0.  ],
        ...,
        [ 0.  ,  1.  , 22.  , ...,  0.  ,  0.  ,  0.07],
        [ 0.  ,  2.  , 11.  , ...,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  1.  , 52.  , ...,  0.  ,  0.  ,  0.44]], dtype=float32))

In [67]:
# fit the autoencoder model to reconstruct input
# history = model.fit(X_train, Y_train, epochs=100, batch_size=16, verbose=2, validation_data=(X_test,Y_test))


Epoch 1/100
7874/7874 - 30s - loss: 0.0111 - val_loss: 0.5783 - 30s/epoch - 4ms/step
Epoch 2/100
7874/7874 - 30s - loss: 0.0118 - val_loss: 0.5510 - 30s/epoch - 4ms/step
Epoch 3/100
7874/7874 - 30s - loss: 0.0117 - val_loss: 0.3299 - 30s/epoch - 4ms/step
Epoch 4/100
7874/7874 - 32s - loss: 0.0116 - val_loss: 0.5620 - 32s/epoch - 4ms/step
Epoch 5/100
7874/7874 - 32s - loss: 0.0110 - val_loss: 0.5926 - 32s/epoch - 4ms/step
Epoch 6/100
7874/7874 - 32s - loss: 0.0102 - val_loss: 0.5615 - 32s/epoch - 4ms/step
Epoch 7/100
7874/7874 - 32s - loss: 0.0107 - val_loss: 0.4951 - 32s/epoch - 4ms/step
Epoch 8/100
7874/7874 - 31s - loss: 0.0103 - val_loss: 0.4896 - 31s/epoch - 4ms/step
Epoch 9/100
7874/7874 - 30s - loss: 0.0098 - val_loss: 0.5815 - 30s/epoch - 4ms/step
Epoch 10/100
7874/7874 - 31s - loss: 0.0098 - val_loss: 0.5761 - 31s/epoch - 4ms/step
Epoch 11/100
7874/7874 - 30s - loss: 0.0101 - val_loss: 0.5311 - 30s/epoch - 4ms/step
Epoch 12/100
7874/7874 - 31s - loss: 0.0109 - val_loss: 0.5787 

In [68]:
# define an encoder model (without the decoder)
encoder = Model(inputs=input_data_shape, outputs=bottleneck)
# save the encoder to file
encoder.save('encoder.h5')
from google.colab import files
files.download("encoder.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Load encoder file and use for random forest and check accuracy, compare with without the h5 file

In [69]:
#Building a Base Model to compare the performance after compressing the data using Encoder model.

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# define model
model = LogisticRegression()
# fit model on training set
model.fit(X_train, Y_train)
# make prediction on test set
yhat = model.predict(X_test)
# calculate accuracy
acc = accuracy_score(Y_test, yhat)
print("Logistic Regression accuracy",acc)

# define model
model = RandomForestClassifier()
# fit model on training set
model.fit(X_train, Y_train)
# make prediction on test set
yhat = model.predict(X_test)
# calculate accuracy
acc = accuracy_score(Y_test, yhat)
print("Random Forest accuracy",acc)

Logistic Regression accuracy 0.7103442157558553
Random Forest accuracy 0.7755056777856636


In [70]:
from google.colab import files
uploaded = files.upload()

Saving encoder.h5 to encoder (1).h5


In [71]:
#Compressing the input data using Encoder Model and fitting it on the Logistic Regression model.
# load the model from file
encoder = load_model('encoder.h5')

# encode the train data
X_train_encode = encoder.predict(X_train)
# encode the test data
X_test_encode = encoder.predict(X_test)
# define the model
model = LogisticRegression(max_iter=1000)
# fit the model on the training set
model.fit(X_train_encode, Y_train)
# make predictions on the test set
yhat = model.predict(X_test_encode)
# calculate classification accuracy
acc = accuracy_score(Y_test, yhat)
print("Logistic Regression accuracy after encoding",acc)

# define the model
model = RandomForestClassifier()
# fit the model on the training set
model.fit(X_train_encode, Y_train)
# make predictions on the test set
yhat = model.predict(X_test_encode)
# calculate classification accuracy
acc = accuracy_score(Y_test, yhat)
print("RandomForest Classifier accuracy after encoding",acc)

Logistic Regression accuracy after encoding 0.43066891412349184
RandomForest Classifier accuracy after encoding 0.7802075940383251


In [76]:
# classification matrix
# Accuracy, Precision, Recall, False Alarm, F-score

tn, fp, fn, tp = confusion_matrix(Y_test, yhat).ravel()
tn, fp, fn, tp

(8142, 4691, 264, 9447)

In [80]:
from pandas.core.indexes.base import trim_front
accuracy_man = (tp+fn)/(tp+fn+fp+fn)
print(accuracy_man)

precision = tp/(tp+fp)
print(precision)

recall = fp/(tp+fn)
print(recall)

false_alarm  = fp/(tp+fn)
print(false_alarm)

f_score = 2*((precision*recall)/(precision+recall))
print(f_score)

0.662143733806082
0.6681991795161975
0.4830604469158686
0.4830604469158686
0.5607433577537305
